In [ ]:
import requests
import configparser
import pandas as pd
import requests
import sqlite3
import time

In [ ]:
def sqlite_connection():
    
    return sqlite3.connect('../data/database/ai_systems')

In [ ]:
def get_apikey():
    
    config = configparser.ConfigParser()
    config.read('../config.ini')
    
    return config['DEFAULT']['APIKEY']

In [ ]:
def get_symbol_list():
    
    df = pd.read_csv('../data/static/sandp100.csv')
    
    return df['Ticker'].tolist()

In [ ]:
def build_dataset():
    
    api_key = get_apikey()
    symbol_list = get_symbol_list()
    con = sqlite_connection()
    
    for symbol in symbol_list:
        
        time.sleep(10)
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={api_key}"
        
        try:
            
            print(f"Parsing {symbol}")
            response = requests.get(url)
            json = response.json()['Weekly Time Series']

            df = pd.DataFrame.from_dict(json, orient='index')
            df.columns = [col.split(' ')[1] for col in df.columns]
            df = df.reset_index()
            df.rename(columns = {'index': 'date'}, inplace = True)
            df['ticker'] = symbol

            df.to_sql(name='weekly_prices', if_exists='append', index=False, con=con)
        
        except:
            print(f"Failed: {url}")


In [ ]:
build_dataset()